In [74]:
from shared_code.utility.spark.set_environ import *

set_azure_env()

spark = get_session("normalize-paths")

In [75]:
from shared_code.utility.storage.table import TableAdapter

table_name = "training"
table_broker: TableAdapter = TableAdapter()

raw_data = table_broker.get_all_entities(table_name)

In [76]:
from shared_code.utility.schemas.spark_table_schema import *

df = spark.createDataFrame(raw_data, schema=image_table_schema)

df.createOrReplaceTempView("temp_training")

In [77]:
import os
from pyspark.sql.functions import udf

@udf(returnType=StringType())
def get_missing_name(path):
	try:
		return os.path.split(path)[-1]
	except:
		return ""

In [78]:
import os
from pyspark.sql.functions import udf
@udf(returnType=StringType())
def get_correct_path(path, sub, name):
	normal_path = os.path.normpath(path)
	if "reddit_images" in normal_path:
		updated_path = os.path.join("D:", os.sep, "data", "images", sub, name)
		return updated_path
	if "raw_reddit_images" in normal_path:
		updated_path = os.path.join("D:", os.sep, "data", "images", sub, name)
		return updated_path
	else:
		updated_path = os.path.join("D:", os.sep, "data", "images", sub, name)
		return updated_path

In [79]:
import os
from pyspark.sql.functions import udf
@udf(returnType=BooleanType())
def check_if_exists(path):
	return os.path.exists(path)

In [80]:
import os
from pyspark.sql.functions import udf
@udf(returnType=BooleanType())
def check_if_curated(curated: bool):
	if curated is None:
		return False
	if curated == "True":
		return True
	else:
		return False

In [81]:
import os
from pyspark.sql.functions import udf
@udf(returnType=StringType())
def get_thumbnail_path(path:str, sub:str, name:str):
	if path == "" or path is None or path == "None" or path == "NaN":
		expected_path = os.path.join("D:", os.sep, "data", "images", sub, "thumbnails", name)
		if os.path.exists(expected_path):
			print(f"Found thumbnail as expected: {expected_path}")
			return expected_path
		else:
			return ""
	else:
		if os.path.exists(path):
			print(f"Found thumbnail already: {path}")
			return path
		else:
			return ""

In [82]:
missing_names_df = spark.sql("""
	select * from temp_training
	where image_name is null
""")

total_missing_image_names = missing_names_df.count()
print(f"Total missing image names: {total_missing_image_names}")

Total missing image names: 0


In [83]:
named_frame = missing_names_df.withColumn("image_name", get_missing_name("image"))

display(named_frame.limit(10).toPandas())

,PartitionKey,RowKey,image,text,id,author,url,flair,permalink,hash,subreddit,caption,exists,image_name,updated_caption,small_image,curated


In [84]:
records = named_frame.toPandas().to_dict('records')

total = len(records)
print(f"Total records to update: {total}")

if total > 0:
	for i, record in enumerate(records):
		if i % 100 == 0:
			print(f"Updating record {i} of {total}")
		table_broker.upsert_entity_to_table(table_name, record)
else:
	print("No records to update")

Total records to update: 0
No records to update


In [85]:
entities = table_broker.get_all_entities(table_name)
data_frame = spark.createDataFrame(entities, schema=image_table_schema)
display(data_frame.limit(10).toPandas())

,PartitionKey,RowKey,image,text,id,author,url,flair,permalink,hash,subreddit,caption,exists,image_name,updated_caption,small_image,curated
0,training,1000cej,D:\data\images\CityPorn\4emw5uldib9a1.jpg,New York in the fog,1000cej,OtterlyFoxy,https://i.redd.it/4emw5uldib9a1.jpg,NaN,/r/CityPorn/comments/1000cej/new_york_in_the_fog/,7a8d96e378c15c8ab8440ac311f12c11,CityPorn,A person is riding a bicycle on a city street.,True,4emw5uldib9a1.jpg,NaN,D:\data\images\CityPorn\thumbnail\4emw5uldib9a...,False
1,training,1000d16,D:\data\images\SFWNextDoorGirls\GgFEagO.jpg,Thoughts about my NYE party outfit?,1000d16,princessxo699,https://i.imgur.com/GgFEagO.jpg,Outfit of the Day,/r/SFWNextDoorGirls/comments/1000d16/thoughts_...,9951b4f82caeb8ba2bd9f79f8d422450,SFWNextDoorGirls,A little girl in a pink dress is holding a pin...,True,GgFEagO.jpg,NaN,D:\data\images\SFWNextDoorGirls\thumbnail\GgFE...,False
2,training,1000fg0,D:\data\images\HotGirlNextDoor\nwa7hts2jb9a1.jpg,(IKTR),1000fg0,BlkBrd1312,https://i.redd.it/nwa7hts2jb9a1.jpg,HotGirlNextDoor,/r/HotGirlNextDoor/comments/1000fg0/iktr/,be8dd55e34216bec1e15e03fa296eacc,HotGirlNextDoor,A woman in a pink bathing suit holds a baby in...,True,nwa7hts2jb9a1.jpg,a woman in a bathing suit with a cell phone in...,D:\data\images\HotGirlNextDoor\thumbnail\nwa7h...,False
3,training,1000glf,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,Just looking for entertainment,1000glf,toolate_sharkbait,https://i.redd.it/4xyb1vgbjb9a1.jpg,Female (18+),/r/AmIhotAF/comments/1000glf/just_looking_for_...,e554c1ed7ffa2740436ac082068b2824,AmIhotAF,A woman in a pink dress is smiling at the camera.,True,4xyb1vgbjb9a1.jpg,NaN,D:\data\images\AmIhotAF\thumbnail\4xyb1vgbjb9a...,False
4,training,1000qiq,D:\data\images\SFWRedheads\0mgrt95plb9a1.jpg,Who’s going to be my new years kiss tonight? 🤧,1000qiq,fireybubbulegum,https://i.redd.it/0mgrt95plb9a1.jpg,NaN,/r/SFWRedheads/comments/1000qiq/whos_going_to_...,7b0be38a08e3316051c6e5a3753bc791,SFWRedheads,A woman in a red dress is talking to a man in ...,False,0mgrt95plb9a1.jpg,NaN,None,False
5,training,1001fon,D:\data\images\SFWRedheads\gp4fhsdorb9a1.jpg,honk if you like my sweater 😁💕,1001fon,glam_pire,https://i.redd.it/gp4fhsdorb9a1.jpg,NaN,/r/SFWRedheads/comments/1001fon/honk_if_you_li...,feed678c3b74ed1d0b7011f28e8f283f,SFWRedheads,A woman in a red dress is talking to a man in ...,False,gp4fhsdorb9a1.jpg,NaN,None,False
6,training,1001ghp,D:\data\images\sfwpetite\v3ra9g4vrb9a1.jpg,Braids,1001ghp,eromorphic,https://i.redd.it/v3ra9g4vrb9a1.jpg,NaN,/r/sfwpetite/comments/1001ghp/braids/,16ca1536c4f6936d3e994df7cb40ab8e,sfwpetite,A girl in a pink bathing suit walks down the b...,True,v3ra9g4vrb9a1.jpg,a beautiful young woman in a bikini posing for...,D:\data\images\sfwpetite\thumbnail\v3ra9g4vrb9...,False
7,training,1001ih5,D:\data\images\SFWNextDoorGirls\bhigr5rbsb9a1.jpg,New Year's Elf,1001ih5,ms_twine_grace,https://i.redd.it/bhigr5rbsb9a1.jpg,NaN,/r/SFWNextDoorGirls/comments/1001ih5/new_years...,9ff18e9afb7b566c0d50add45cbfb989,SFWNextDoorGirls,A little girl in a pink dress is dancing on a ...,False,bhigr5rbsb9a1.jpg,NaN,None,False
8,training,1001kp4,D:\data\images\AmIhotAF\dn7lcclvsb9a1.jpg,Happy New Years Eve,1001kp4,Hotblondie69420,https://i.redd.it/dn7lcclvsb9a1.jpg,Female (18+),/r/AmIhotAF/comments/1001kp4/happy_new_years_eve/,7137ef4fe19a97c757553bfa62b23a33,AmIhotAF,A woman in a white dress is talking to a man i...,False,dn7lcclvsb9a1.jpg,NaN,None,False
9,training,1001ssd,D:\data\images\sfwpetite\ldz8tgpsub9a1.jpg,Incredible body,1001ssd,CertifiedDrinkSpill,https://i.redd.it/ldz8tgpsub9a1.jpg,NaN,/r/sfwpetite/comments/1001ssd/incredible_body/,a0b0ffef01a7abf9de351000026d49f0,sfwpetite,A woman in a bikini and a man in a white t-shi...,True,ldz8tgpsub9a1.jpg,NaN,D:\data\images\sfwpetite\thumbnail\ldz8tgpsub9...,False


In [86]:
new_df = data_frame\
	.withColumn("updated_path", get_correct_path("image", "subreddit", "image_name"))\
	.withColumn("exists", check_if_exists("updated_path"))\
	.withColumn("curated", check_if_curated("curated"))\
	.withColumn("expected_small_image", get_thumbnail_path("small_image", "subreddit", "image_name"))\
	.drop("exits")\
	.drop("image")\
	.withColumnRenamed("updated_path", "image")\
	.drop("small_image")\
	.withColumnRenamed("expected_small_image", "small_image")

pandas_frame = new_df.toPandas()

display(pandas_frame)

,PartitionKey,RowKey,text,id,author,url,flair,permalink,hash,subreddit,caption,exists,image_name,updated_caption,curated,image,small_image
0,training,1000cej,New York in the fog,1000cej,OtterlyFoxy,https://i.redd.it/4emw5uldib9a1.jpg,NaN,/r/CityPorn/comments/1000cej/new_york_in_the_fog/,7a8d96e378c15c8ab8440ac311f12c11,CityPorn,A person is riding a bicycle on a city street.,True,4emw5uldib9a1.jpg,NaN,False,D:\data\images\CityPorn\4emw5uldib9a1.jpg,D:\data\images\CityPorn\thumbnail\4emw5uldib9a...
1,training,1000d16,Thoughts about my NYE party outfit?,1000d16,princessxo699,https://i.imgur.com/GgFEagO.jpg,Outfit of the Day,/r/SFWNextDoorGirls/comments/1000d16/thoughts_...,9951b4f82caeb8ba2bd9f79f8d422450,SFWNextDoorGirls,A little girl in a pink dress is holding a pin...,True,GgFEagO.jpg,NaN,False,D:\data\images\SFWNextDoorGirls\GgFEagO.jpg,D:\data\images\SFWNextDoorGirls\thumbnail\GgFE...
2,training,1000fg0,(IKTR),1000fg0,BlkBrd1312,https://i.redd.it/nwa7hts2jb9a1.jpg,HotGirlNextDoor,/r/HotGirlNextDoor/comments/1000fg0/iktr/,be8dd55e34216bec1e15e03fa296eacc,HotGirlNextDoor,A woman in a pink bathing suit holds a baby in...,True,nwa7hts2jb9a1.jpg,a woman in a bathing suit with a cell phone in...,False,D:\data\images\HotGirlNextDoor\nwa7hts2jb9a1.jpg,D:\data\images\HotGirlNextDoor\thumbnail\nwa7h...
3,training,1000glf,Just looking for entertainment,1000glf,toolate_sharkbait,https://i.redd.it/4xyb1vgbjb9a1.jpg,Female (18+),/r/AmIhotAF/comments/1000glf/just_looking_for_...,e554c1ed7ffa2740436ac082068b2824,AmIhotAF,A woman in a pink dress is smiling at the camera.,True,4xyb1vgbjb9a1.jpg,NaN,False,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,D:\data\images\AmIhotAF\thumbnail\4xyb1vgbjb9a...
4,training,1000qiq,Who’s going to be my new years kiss tonight? 🤧,1000qiq,fireybubbulegum,https://i.redd.it/0mgrt95plb9a1.jpg,NaN,/r/SFWRedheads/comments/1000qiq/whos_going_to_...,7b0be38a08e3316051c6e5a3753bc791,SFWRedheads,A woman in a red dress is talking to a man in ...,False,0mgrt95plb9a1.jpg,NaN,False,D:\data\images\SFWRedheads\0mgrt95plb9a1.jpg,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6496,training,zzy49v,"Ronda's bridge is brutal, Spain",zzy49v,Withalstyle968,https://i.redd.it/dwu7i1dni99a1.jpg,NaN,/r/CityPorn/comments/zzy49v/rondas_bridge_is_b...,02ae3631262b2de84b0b24c4275deae9,CityPorn,A group of people are standing in front of a l...,True,dwu7i1dni99a1.jpg,NaN,False,D:\data\images\CityPorn\dwu7i1dni99a1.jpg,D:\data\images\CityPorn\thumbnail\dwu7i1dni99a...
6497,training,zzyerp,Good evening everyone ❤️,zzyerp,Juliadomaina,https://i.redd.it/hgktc1on2b9a1.jpg,Female (18+),/r/AmIhotAF/comments/zzyerp/good_evening_every...,9279353b1761d62e7f8ca587a4146abf,AmIhotAF,A girl in a red jacket is sitting on a red horse.,True,hgktc1on2b9a1.jpg,NaN,False,D:\data\images\AmIhotAF\hgktc1on2b9a1.jpg,D:\data\images\AmIhotAF\thumbnail\hgktc1on2b9a...
6498,training,zzzg13,Cute but Deadly 😋,zzzg13,AdConsistent3184,https://i.redd.it/3rv0ql34bb9a1.jpg,NaN,/r/SFWRedheads/comments/zzzg13/cute_but_deadly/,ba89401054b09d3ff2b608705f8a7110,SFWRedheads,A little girl in a pink dress is sitting on a ...,False,3rv0ql34bb9a1.jpg,NaN,False,D:\data\images\SFWRedheads\3rv0ql34bb9a1.jpg,
6499,training,zzzlyl,"Taipei, Taiwan",zzzlyl,Particular-Chip2355,https://upload.wikimedia.org/wikipedia/commons...,NaN,/r/CityPorn/comments/zzzlyl/taipei_taiwan/,3cac9b29f27d5fc26ddcec4e1641649f,CityPorn,Error in captioning image,True,Minquan_Bridge2017_TAIWAN.jpg,NaN,False,D:\data\images\CityPorn\Minquan_Bridge2017_TAI...,


In [87]:
writable_df = spark.createDataFrame(pandas_frame.to_dict(orient='records'), schema=image_table_schema)
print(f"Total records to write: {writable_df.count()}")
display(writable_df.limit(10).toPandas())
writable_df.write.parquet("D:\\data\\processed\\reddit_images.parquet", mode="overwrite")

Total records to write: 6501


,PartitionKey,RowKey,image,text,id,author,url,flair,permalink,hash,subreddit,caption,exists,image_name,updated_caption,small_image,curated
0,training,1000cej,D:\data\images\CityPorn\4emw5uldib9a1.jpg,New York in the fog,1000cej,OtterlyFoxy,https://i.redd.it/4emw5uldib9a1.jpg,NaN,/r/CityPorn/comments/1000cej/new_york_in_the_fog/,7a8d96e378c15c8ab8440ac311f12c11,CityPorn,A person is riding a bicycle on a city street.,True,4emw5uldib9a1.jpg,NaN,D:\data\images\CityPorn\thumbnail\4emw5uldib9a...,False
1,training,1000d16,D:\data\images\SFWNextDoorGirls\GgFEagO.jpg,Thoughts about my NYE party outfit?,1000d16,princessxo699,https://i.imgur.com/GgFEagO.jpg,Outfit of the Day,/r/SFWNextDoorGirls/comments/1000d16/thoughts_...,9951b4f82caeb8ba2bd9f79f8d422450,SFWNextDoorGirls,A little girl in a pink dress is holding a pin...,True,GgFEagO.jpg,NaN,D:\data\images\SFWNextDoorGirls\thumbnail\GgFE...,False
2,training,1000fg0,D:\data\images\HotGirlNextDoor\nwa7hts2jb9a1.jpg,(IKTR),1000fg0,BlkBrd1312,https://i.redd.it/nwa7hts2jb9a1.jpg,HotGirlNextDoor,/r/HotGirlNextDoor/comments/1000fg0/iktr/,be8dd55e34216bec1e15e03fa296eacc,HotGirlNextDoor,A woman in a pink bathing suit holds a baby in...,True,nwa7hts2jb9a1.jpg,a woman in a bathing suit with a cell phone in...,D:\data\images\HotGirlNextDoor\thumbnail\nwa7h...,False
3,training,1000glf,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,Just looking for entertainment,1000glf,toolate_sharkbait,https://i.redd.it/4xyb1vgbjb9a1.jpg,Female (18+),/r/AmIhotAF/comments/1000glf/just_looking_for_...,e554c1ed7ffa2740436ac082068b2824,AmIhotAF,A woman in a pink dress is smiling at the camera.,True,4xyb1vgbjb9a1.jpg,NaN,D:\data\images\AmIhotAF\thumbnail\4xyb1vgbjb9a...,False
4,training,1000qiq,D:\data\images\SFWRedheads\0mgrt95plb9a1.jpg,Who’s going to be my new years kiss tonight? 🤧,1000qiq,fireybubbulegum,https://i.redd.it/0mgrt95plb9a1.jpg,NaN,/r/SFWRedheads/comments/1000qiq/whos_going_to_...,7b0be38a08e3316051c6e5a3753bc791,SFWRedheads,A woman in a red dress is talking to a man in ...,False,0mgrt95plb9a1.jpg,NaN,,False
5,training,1001fon,D:\data\images\SFWRedheads\gp4fhsdorb9a1.jpg,honk if you like my sweater 😁💕,1001fon,glam_pire,https://i.redd.it/gp4fhsdorb9a1.jpg,NaN,/r/SFWRedheads/comments/1001fon/honk_if_you_li...,feed678c3b74ed1d0b7011f28e8f283f,SFWRedheads,A woman in a red dress is talking to a man in ...,False,gp4fhsdorb9a1.jpg,NaN,,False
6,training,1001ghp,D:\data\images\sfwpetite\v3ra9g4vrb9a1.jpg,Braids,1001ghp,eromorphic,https://i.redd.it/v3ra9g4vrb9a1.jpg,NaN,/r/sfwpetite/comments/1001ghp/braids/,16ca1536c4f6936d3e994df7cb40ab8e,sfwpetite,A girl in a pink bathing suit walks down the b...,True,v3ra9g4vrb9a1.jpg,a beautiful young woman in a bikini posing for...,D:\data\images\sfwpetite\thumbnail\v3ra9g4vrb9...,False
7,training,1001ih5,D:\data\images\SFWNextDoorGirls\bhigr5rbsb9a1.jpg,New Year's Elf,1001ih5,ms_twine_grace,https://i.redd.it/bhigr5rbsb9a1.jpg,NaN,/r/SFWNextDoorGirls/comments/1001ih5/new_years...,9ff18e9afb7b566c0d50add45cbfb989,SFWNextDoorGirls,A little girl in a pink dress is dancing on a ...,False,bhigr5rbsb9a1.jpg,NaN,,False
8,training,1001kp4,D:\data\images\AmIhotAF\dn7lcclvsb9a1.jpg,Happy New Years Eve,1001kp4,Hotblondie69420,https://i.redd.it/dn7lcclvsb9a1.jpg,Female (18+),/r/AmIhotAF/comments/1001kp4/happy_new_years_eve/,7137ef4fe19a97c757553bfa62b23a33,AmIhotAF,A woman in a white dress is talking to a man i...,False,dn7lcclvsb9a1.jpg,NaN,,False
9,training,1001ssd,D:\data\images\sfwpetite\ldz8tgpsub9a1.jpg,Incredible body,1001ssd,CertifiedDrinkSpill,https://i.redd.it/ldz8tgpsub9a1.jpg,NaN,/r/sfwpetite/comments/1001ssd/incredible_body/,a0b0ffef01a7abf9de351000026d49f0,sfwpetite,A woman in a bikini and a man in a white t-shi...,True,ldz8tgpsub9a1.jpg,NaN,D:\data\images\sfwpetite\thumbnail\ldz8tgpsub9...,False


In [88]:
processed = spark.read.parquet("D:\\data\\processed\\reddit_images.parquet")
display(processed.limit(10).toPandas())

,PartitionKey,RowKey,image,text,id,author,url,flair,permalink,hash,subreddit,caption,exists,image_name,updated_caption,small_image,curated
0,training,11090tg,D:\data\images\EarthPorn\vc1wh65itqha1.jpg,"Depoe Bay, OR USA [OC] 1080x1920",11090tg,Bpercell,https://i.redd.it/vc1wh65itqha1.jpg,NaN,/r/EarthPorn/comments/11090tg/depoe_bay_or_usa...,31c6df5a2815d8e5e8c85fe51b08460e,EarthPorn,A group of people are sitting on the shore of ...,True,vc1wh65itqha1.jpg,NaN,D:\data\images\EarthPorn\thumbnail\vc1wh65itqh...,False
1,training,11092s0,D:\data\images\SFWRedheads\8lb4z8n3uqha1.jpg,Morning hair xx,11092s0,demidiamond4,https://i.redd.it/8lb4z8n3uqha1.jpg,NaN,/r/SFWRedheads/comments/11092s0/morning_hair_xx/,afe1ae1ae3e663b0ca62cb3b19a5c3ef,SFWRedheads,A girl in a pink dress is smiling.,True,8lb4z8n3uqha1.jpg,NaN,D:\data\images\SFWRedheads\thumbnail\8lb4z8n3u...,False
2,training,110961d,D:\data\images\CityPorn\2czgh0e5vqha1.jpg,St Paul’s Cathedral. London.,110961d,OkCardiologist2765,https://i.redd.it/2czgh0e5vqha1.jpg,NaN,/r/CityPorn/comments/110961d/st_pauls_cathedra...,ef3c3f82ec490f8d3ccc06948258a3b4,CityPorn,A man in a red coat is standing in front of a ...,True,2czgh0e5vqha1.jpg,NaN,D:\data\images\CityPorn\thumbnail\2czgh0e5vqha...,False
3,training,1109a10,D:\data\images\SFWRedheads\hy9j0og8wqha1.jpg,Do you like my natural red curls?!,1109a10,ketunicorn,https://i.redd.it/hy9j0og8wqha1.jpg,NaN,/r/SFWRedheads/comments/1109a10/do_you_like_my...,9008fe70d17b904fcfc7c6a2232ad569,SFWRedheads,A girl in a pink dress is standing in front of...,True,hy9j0og8wqha1.jpg,NaN,D:\data\images\SFWRedheads\thumbnail\hy9j0og8w...,False
4,training,1109ycm,D:\data\images\CityPorn\7beed5n54rha1.jpg,"Taipei, the Seoul of Taiwan (which makes Kaohs...",1109ycm,Ducky118,https://i.redd.it/7beed5n54rha1.jpg,NaN,/r/CityPorn/comments/1109ycm/taipei_the_seoul_...,848b795097dca410c409797c679ab4fa,CityPorn,A bird flies over the water.,True,7beed5n54rha1.jpg,NaN,D:\data\images\CityPorn\thumbnail\7beed5n54rha...,False
5,training,110b0wt,D:\data\images\SFWNextDoorGirls\6piZs7j.jpg,"A real no makeup moment, cause this is how Sun...",110b0wt,princessxo699,https://i.imgur.com/6piZs7j.jpg,Feelin Happy 😃,/r/SFWNextDoorGirls/comments/110b0wt/a_real_no...,5ccd774afb17fb0c44460121ed9ce61d,SFWNextDoorGirls,A girl in a pink hat and sunglasses is talking...,True,6piZs7j.jpg,NaN,D:\data\images\SFWNextDoorGirls\thumbnail\6piZ...,False
6,training,110b1an,D:\data\images\CityPorn\rgotpnatzpha1.jpg,"Duisburg, Detroit of Germany",110b1an,Expensive-Team7416,https://i.redd.it/rgotpnatzpha1.jpg,NaN,/r/CityPorn/comments/110b1an/duisburg_detroit_...,3eb930bdf3fac4e5b16617edad9df65a,CityPorn,A bird flies over the water.,True,rgotpnatzpha1.jpg,NaN,D:\data\images\CityPorn\thumbnail\rgotpnatzpha...,False
7,training,110bipg,D:\data\images\SFWNextDoorGirls\xywgncrbnrha1.jpg,enjoy your Sunday!,110bipg,trickyalice,https://i.redd.it/xywgncrbnrha1.jpg,Outfit of the Day,/r/SFWNextDoorGirls/comments/110bipg/enjoy_you...,9475adbe05427c47772346f8ab672952,SFWNextDoorGirls,A woman in a pink shirt and jeans is talking t...,True,xywgncrbnrha1.jpg,NaN,D:\data\images\SFWNextDoorGirls\thumbnail\xywg...,False
8,training,110c2gi,D:\data\images\SFWRedheads\99hm504mtrha1.jpg,Happy sunday! Sending greetings! ❤️,110c2gi,GingeMariia,https://i.redd.it/99hm504mtrha1.jpg,NaN,/r/SFWRedheads/comments/110c2gi/happy_sunday_s...,20ced2827eb9e9cd3464835306cfd075,SFWRedheads,A girl in a pink shirt and blue jeans is stand...,True,99hm504mtrha1.jpg,NaN,D:\data\images\SFWRedheads\thumbnail\99hm504mt...,False
9,training,110c2vo,D:\data\images\SFWRedheads\kkerxboqtrha1.jpg,The kind of girl you can have a beer with.,110c2vo,MissKatiesss,https://i.redd.it/kkerxboqtrha1.jpg,NaN,/r/SFWRedheads/comments/110c2vo/the_kind_of_gi...,f53572296a0b8fb6654808866440316a,SFWRedheads,A woman in a red dress is talking to a man in ...,True,kkerxboqtrha1.jpg,NaN,D:\data\images\SFWRedheads\thumbnail\kkerxboqt...,False
